In [1]:
import threading
import time
import numpy as np
import pandas as pd
from pynput import mouse
from screeninfo import get_monitors
import joblib
from sklearn.preprocessing import StandardScaler
from features import compute_features
from collections import defaultdict

window_size = 50  # Number of timestamps (e.g., 50 * 0.1s = 5 seconds)
interval = 1  # Interval to process events (in seconds)

classification_classes = {
  "web_browsing": 0, # web
  # "ppt": 1, # design tools,
  # "figma": 1, # design tools
  # "reading": 2, # reading papers
  "finding_mines": 3, # game
  "chess": 3, # game
  "youtube": 4, # youtube
  # "chatting": 5, # chatting
}

classification_map = defaultdict(str)
for key, value in classification_classes.items():
    classification_map[value] += key + ", "

# Load the pretrained model and scaler
model = joblib.load('boosting_model.pkl')
scaler = joblib.load('boosting_scaler.pkl')  # Assuming the scaler was saved separately

# Get monitor information
monitors = get_monitors()
monitor = monitors[0]  # Assume the primary monitor
width, height = monitor.width, monitor.height

# Global variables
event_list = []
lock = threading.Lock()


def adjust_coordinates(x, y):
    x_adj = (x - monitor.x) / width
    y_adj = (y - monitor.y) / height
    return x_adj, y_adj

def on_move(x, y):
    x_adj, y_adj = adjust_coordinates(x, y)
    with lock:
        event_list.append(('move', time.time(), x_adj, y_adj))

def on_click(x, y, button, pressed):
    x_adj, y_adj = adjust_coordinates(x, y)
    with lock:
        event_list.append(('click', time.time(), x_adj, y_adj, button.name, pressed))

def on_scroll(x, y, dx, dy):
    x_adj, y_adj = adjust_coordinates(x, y)
    with lock:
        event_list.append(('scroll', time.time(), x_adj, y_adj, dx, dy))

def process_events():
    """Process events at regular intervals to make live predictions."""
    while True:
        time.sleep(interval)
        current_time = time.time()
        with lock:
          # Keep events within the last window_size * 0.1 seconds
          window_start_time = current_time - window_size * 0.1
          recent_events = [event for event in event_list if event[1] >= window_start_time]
          # Remove old events
          event_list[:] = recent_events

        # Convert events to DataFrame
        event_dicts = []
        for event in recent_events:
            if event[0] == 'move':
                event_dict = {'event_type': 'move', 'time': event[1], 'x': event[2], 'y': event[3]}
            elif event[0] == 'click':
                event_dict = {'event_type': 'click', 'time': event[1], 'x': event[2], 'y': event[3],
                              'button': event[4], 'pressed': event[5]}
            elif event[0] == 'scroll':
                event_dict = {'event_type': 'scroll', 'time': event[1], 'x': event[2], 'y': event[3],
                              'dx': event[4], 'dy': event[5]}
            else:
                continue
            event_dicts.append(event_dict)

        df = pd.DataFrame(event_dicts, columns=['event_type', 'time', 'x', 'y', 'button', 'pressed', 'dx', 'dy'])
        df = df.sort_values('time').reset_index(drop=True)

        # Organize events into time windows
        df['timestamp'] = ((df['time'] - window_start_time) // 0.1).astype(int)

        # Fill missing timestamps
        unique_timestamps = df['timestamp'].unique()
        full_range = np.arange(0, window_size)
        missing_timestamps = np.setdiff1d(full_range, unique_timestamps)

        if missing_timestamps.size > 0:
            empty_rows = pd.DataFrame({'timestamp': missing_timestamps})
            df = pd.concat([df, empty_rows], ignore_index=True)
            df = df.sort_values('timestamp').reset_index(drop=True).fillna(0)

        
        features_df = df.groupby('timestamp').apply(compute_features, include_groups=False).reset_index(drop=True)
        # Preprocess features
        feature_array = features_df.values.reshape(1, -1)
        feature_array = np.nan_to_num(feature_array, posinf=np.finfo(np.float32).max, neginf=np.finfo(np.float32).min)
        feature_array = scaler.transform(feature_array)

        # Make prediction
        prediction = model.predict(feature_array)
        # Optionally, get prediction probabilities
        # probabilities = model.predict_proba(feature_array)

        # Output prediction
        print(f"Prediction: {classification_map[prediction[0]]}")

def main():
  # Start the processing thread
  processing_thread = threading.Thread(target=process_events)
  processing_thread.daemon = True
  processing_thread.start()

  # Start the mouse listener
  with mouse.Listener(on_move=on_move, on_click=on_click, on_scroll=on_scroll) as listener:
      listener.join()

if __name__ == "__main__":
    main()

Prediction: finding_mines, chess, 
Prediction: finding_mines, chess, 
Prediction: finding_mines, chess, 
Prediction: youtube, 
Prediction: finding_mines, chess, 
Prediction: web_browsing, 
Prediction: web_browsing, 
Prediction: web_browsing, 
Prediction: web_browsing, 
Prediction: web_browsing, 
Prediction: web_browsing, 
Prediction: web_browsing, 
Prediction: web_browsing, 
Prediction: finding_mines, chess, 
Prediction: web_browsing, 
Prediction: web_browsing, 
Prediction: youtube, 
Prediction: web_browsing, 
Prediction: web_browsing, 


/var/folders/df/fxnhsyqj16v2x3xg031vlyyr0000gn/T/ipykernel_48783/284142215.py:105: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.sort_values('timestamp').reset_index(drop=True).fillna(0)


Prediction: youtube, 


/var/folders/df/fxnhsyqj16v2x3xg031vlyyr0000gn/T/ipykernel_48783/284142215.py:105: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.sort_values('timestamp').reset_index(drop=True).fillna(0)


Prediction: youtube, 


...

Prediction: youtube, 


/var/folders/df/fxnhsyqj16v2x3xg031vlyyr0000gn/T/ipykernel_48783/284142215.py:105: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.sort_values('timestamp').reset_index(drop=True).fillna(0)


Prediction: youtube, 


/var/folders/df/fxnhsyqj16v2x3xg031vlyyr0000gn/T/ipykernel_48783/284142215.py:105: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df = df.sort_values('timestamp').reset_index(drop=True).fillna(0)


Prediction: youtube, 
